0912時点　精度：0.85894<br>
452位/17111メンバー

In [ ]:
452/17111

0.026415755946467185

データを読み込み

In [ ]:
import pandas as pd

# 学習用データ
train = pd.read_csv("train.csv",index_col=0)
# 評価用データ
test = pd.read_csv("test.csv",index_col=0)
# 応募用サンプルファイル
sample_submit = pd.read_csv("sample_submit.csv",index_col=0,header=None)

データを結合

In [ ]:
df = pd.concat([test,train],axis = 0,sort = False)
df

,pclass,sex,age,sibsp,parch,fare,embarked,survived
id,,,,,,,,
0,3,male,22.0,1,0,7.2500,S,NaN
1,1,female,38.0,1,0,71.2833,C,NaN
2,3,female,26.0,0,0,7.9250,S,NaN
5,3,male,NaN,0,0,8.4583,Q,NaN
6,1,male,54.0,0,0,51.8625,S,NaN
...,...,...,...,...,...,...,...,...
873,3,male,47.0,0,0,9.0000,S,0.0
874,2,female,28.0,1,0,24.0000,C,1.0
879,1,female,56.0,0,1,83.1583,C,1.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 0 to 888
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    891 non-null    int64  
 1   sex       891 non-null    object 
 2   age       714 non-null    float64
 3   sibsp     891 non-null    int64  
 4   parch     891 non-null    int64  
 5   fare      891 non-null    float64
 6   embarked  889 non-null    object 
 7   survived  445 non-null    float64
dtypes: float64(3), int64(3), object(2)
memory usage: 62.6+ KB


#Family　精度が下がる

**新たな特徴量を作成**<br><br>
Family = SibSp + Parch + 1 を特徴量とし、グルーピング<br>
* SibSpはタイタニックに同乗している兄弟や配偶者の数
* Parchはタイタニックに同乗している親や子供の数

In [ ]:
#「家族の合計人数」 を表す family 列を作成している。
df["family"] = df["sibsp"] + df["parch"] + 1

# 家族が2〜4人なら family_label = 2
df.loc[(df["family"] >= 2) & (df["family"] <= 4),"family_label"] = 2

# 家族が5〜7人 または 1人なら family_label = 1
df.loc[(df["family"] >= 5) & (df["family"] <= 7) | (df["family"] == 1),"family_label"] = 1

# 家族が8人以上なら family_label = 0
df.loc[(df["family"] >= 8),"family_label"] = 0

#fare

「S港から出発して3等客室の人たちの中央値運賃」で、欠損している運賃を補完する [精度変化なし]

In [ ]:
#港が S で、かつ 3等客室 の人たち…df.loc[(df['Embarked'] == 'S') & (df['Pclass'] == 3),
#'Fare' : その人たちの運賃列を取り出す
#.median() : そのグループの運賃の中央値を計算する
fare=df.loc[(df['embarked'] == 'S') & (df['pclass'] == 3), 'fare'].median()
#fillna(fare) : 欠損値 (NaN) のところを、上で計算した中央値で埋める
df['fare']=df['fare'].fillna(fare)

#ageの補完

0.851319⇒0.85894UP

推定に使用する項目を指定

In [ ]:
age_df = df[["age","pclass","sex","parch","sibsp"]]
age_df

,age,pclass,sex,parch,sibsp
id,,,,,
0,22.0,3,male,0,1
1,38.0,1,female,0,1
2,26.0,3,female,0,0
5,NaN,3,male,0,0
6,54.0,1,male,0,0
...,...,...,...,...,...
873,47.0,3,male,0,0
874,28.0,2,female,0,1
879,56.0,1,female,1,0


ラベル特徴量をワンホットエンコーディング

In [ ]:
age_df = pd.get_dummies(age_df)
age_df

,age,pclass,parch,sibsp,sex_female,sex_male
id,,,,,,
0,22.0,3,0,1,False,True
1,38.0,1,0,1,True,False
2,26.0,3,0,0,True,False
5,NaN,3,0,0,False,True
6,54.0,1,0,0,False,True
...,...,...,...,...,...,...
873,47.0,3,0,0,False,True
874,28.0,2,0,1,True,False
879,56.0,1,1,0,True,False


学習データとテストデータに分離し、numpyに変換<br>
scikit-learn の fit() には numpy配列が必要

In [ ]:
#age 列が 欠損ではない行（= 年齢が分かっている人） を know_age
know_age = age_df[age_df.age.notnull()].values
#age 列が 欠損の行（= 年齢が不明な人） を unknown_age
unknown_age = age_df[age_df.age.isnull()].values
#unknown_age.isnull().sum()

学習データをX, yに分離

In [ ]:
x = know_age[:,1:] #説明変数　そのほかの特徴量
y = know_age[:,0] #目的変数　年齢
y = y.astype(int)
y.shape

(714,)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#random_state=0…乱数の種（シード）を固定
#n_estimators=100…作る決定木（木の数）の本数。デフォルトは100。
#n_jobs=1…並列処理するCPUコア数。
model = RandomForestClassifier(random_state=0,n_estimators=100,n_jobs=1)
model.fit(x,y)

RandomForestClassifier(n_jobs=1, random_state=0)

推定モデルを使って、テストデータのAgeを予測し、補完

In [ ]:
#[:, 1::] : 配列の 1列目以降 を使う（つまり age 自体を除いた特徴量だけを使う）
predictAge = model.predict(unknown_age[:,1::])
#df.age.isnull() : age が NaN の行だけを選択
df.loc[(df.age.isnull()),"age"] = predictAge

#学習

In [ ]:
df = pd.get_dummies(df)
df

,pclass,age,sibsp,parch,fare,survived,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
id,,,,,,,,,,,
0,3,22.0,1,0,7.2500,NaN,False,True,False,False,True
1,1,38.0,1,0,71.2833,NaN,True,False,True,False,False
2,3,26.0,0,0,7.9250,NaN,True,False,False,False,True
5,3,21.0,0,0,8.4583,NaN,False,True,False,True,False
6,1,54.0,0,0,51.8625,NaN,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...
873,3,47.0,0,0,9.0000,0.0,False,True,False,False,True
874,2,28.0,1,0,24.0000,1.0,True,False,True,False,False
879,1,56.0,0,1,83.1583,1.0,True,False,True,False,False


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 0 to 888
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   pclass      891 non-null    int64  
 1   age         891 non-null    float64
 2   sibsp       891 non-null    int64  
 3   parch       891 non-null    int64  
 4   fare        891 non-null    float64
 5   survived    445 non-null    float64
 6   sex_female  891 non-null    bool   
 7   sex_male    891 non-null    bool   
 8   embarked_C  891 non-null    bool   
 9   embarked_Q  891 non-null    bool   
 10  embarked_S  891 non-null    bool   
dtypes: bool(5), float64(3), int64(3)
memory usage: 53.1 KB


In [ ]:
train = df[df["survived"].notnull()]
test = df[df["survived"].isnull()]
x_train = train.drop("survived", axis=1)
y_train = train["survived"]

In [ ]:
model = RandomForestClassifier(random_state=0,n_estimators=100,n_jobs=1)
model.fit(x_train,y_train)

RandomForestClassifier(n_jobs=1, random_state=0)

In [ ]:
test_X = test.drop("survived", axis=1, errors="ignore")

In [ ]:
#predict_proba といった分類確信度のスコアを取得できるメソッド
#[:,1]…: は「全行」、
#1 は「2列目（インデックスは0始まりなのでクラス1の確率）」
#クラス1に属する確率だけを取り出す。
pred = model.predict_proba(test_X)[:,1]

各要素は そのサンプルがクラス1である確率。
* 0.105781 → 10.6% の確率でクラス1
* 0.92529693 → 92.5% の確率でクラス1
* 0.64274681 → 64.3% の確率でクラス1
* 0.16363808 → 16.4% の確率でクラス1
* 0.27498188 → 27.5% の確率でクラス1

In [ ]:
pred[:5]

array([0.12      , 0.98      , 0.34      , 0.08805267, 0.25      ])

In [ ]:
print(len(pred))

446


In [ ]:
# 課題が0/1のラベル提出を要求する場合はこちら
sample_submit[1] = (pred >= 0.5).astype(int)
sample_submit.to_csv("submit.csv", index=True, header=False, encoding="utf-8")

In [ ]:
out = pd.DataFrame({0: test.index, 1: pred})
out.to_csv("submit.csv",
           index=False, header=False,
           encoding="utf-8",
           lineterminator="\n",
           float_format="%.10f")

In [ ]:
import pandas as pd
df = pd.read_csv("submit.csv", header=None)
assert df.shape == (446, 2)
assert df[1].between(0, 1).all()
print("OK")

OK


特徴量選択とは、機械学習のモデルを使用する際に有効な特徴量の組み合わせを探索するプロセスのことを表しています。

特徴量の種類
* PassengerId : 乗客のID
* Survived : 生死(0:死亡,1:生存)
* Pclass : 乗客の社会階級(1:Upper, 2:Middle, 3:Lower)
* Name : 名前
* Sex : 性別
* Age : 年齢
* SibSp : 一緒に乗っている兄弟、配偶者の数
* Parch : 一緒に乗っている親、子供の数
* Ticket : チケットの番号
* Fare : 乗船料
* Cabin : 客室番号
* Embarked : 乗船した港